# Chargement des données

In [ ]:
import pandas as pd

from IPython.display import display
from urllib.parse import quote

doc2start = {"bpt6k63243601": 123, "bpt6k62931221": 151, "bpt6k6286466w": 189, "bpt6k6393838j": 219, "bpt6k6331310g": 216, "bpt6k6292987t": 353, "bpt6k62906378": 288, "bpt6k6391515w": 319, "bpt6k6315927h": 349, "bpt6k6319106t": 324, "bpt6k6315985z": 82, "bpt6k63959929": 82, "bpt6k63197984": 56, "bpt6k6389871r": 77, "bpt6k6319811j": 79, "bpt6k6282019m": 72, "bpt6k6314752k": 190, "bpt6k6305463c": 113, "bpt6k6318531z": 108, "bpt6k6324389h": 72, "bpt6k63243920": 80, "bpt6k6309075f": 96, "bpt6k6333200c": 132, "bpt6k63243905": 134, "bpt6k6333170p": 137, "bpt6k96727875": 135, "bpt6k9764746t": 99, "bpt6k97645375": 123, "bpt6k9672117f": 125, "bpt6k9763554c": 123, "bpt6k9763553z": 105, "bpt6k9677392n": 110, "bpt6k9692809v": 113, "bpt6k9762929c": 129, "bpt6k9672776c": 119, "bpt6k9764647w": 121, "bpt6k9669143t": 145, "bpt6k9677737t": 139, "bpt6k9668037f": 167, "bpt6k96839542": 171, "bpt6k96762564": 185, "bpt6k9685861g": 189, "bpt6k9763471j": 153, "bpt6k9762899p": 157, "bpt6k97630871": 11, "bpt6k9684454n": 235, "bpt6k9732740w": 239, "bpt6k9684013b": 189, "bpt6k9692626p": 305, "bpt6k9685098r": 281, "bpt6k9764402m": 329, "bpt6k97631451": 322, "bpt6k9776121t": 49, "bpt6k9775724t": 33, "bpt6k97774838": 327, "bpt6k9780089g": 339}

def entry2url(row):
    """
    Takes a row of an Annuaire csv and
    transforms it to the corresponding Gallica url
    """
    url = "https://gallica.bnf.fr/ark:/12148/"
    
    directory = row['directory']
    page = row['page'] - doc2start[directory]
    url += f"{row['directory']}/f{row['page']-doc2start[row['directory']]}"
    
    r_strings = []
    if 'name' in row and pd.notna(row['name']):
        r_strings.append(quote(row['name'].replace('.', ' ')))
    if 'job' in row and pd.notna(row['job']):
        r_strings.append(quote(row['job'].replace('.', ' ')))
    if 'street' in row and pd.notna(row['street']):
        r_strings.append(quote(row['street'].replace('.', ' ')))
    if 'number' in row and pd.notna(row['number']):
        r_strings.append(quote(row['number'].replace('.', ' ')))
    
    if len(r_strings) > 0:
        url += f".item.r={'%20'.join(r_strings)}.zoom"
    
    return url

def add_clickable_url(bottin_dataframe):
    bottin_dataframe = bottin_dataframe.copy()
    bottin_dataframe['url'] = bottin_dataframe.apply(entry2url, axis=1)
    #def make_clickable(val):
    #    return '<a href="{}">gallica url</a>'.format(val,val)

    #return bottin_dataframe.style.format(make_clickable, subset=['url'])
    return bottin_dataframe

In [ ]:
data_bottin = pd.read_csv('bottin_data_groupe_6.csv')

# Clean

Nettoyer le numéro de rue: garder seulement le premier nombre et éventuellement 'bis'

In [ ]:
data_bottin['number_clean'] = data_bottin['number'].str.extract('(^\d+(?: ?bis)?).*')

Drop duplicated entries

In [ ]:
data_bottin.drop_duplicates(subset =['name', 'year', 'job', 'street_clean', 'number_clean'], keep = 'first', inplace = True)

Ajout des hyperliens en texte brut

In [ ]:
data_bottin = add_clickable_url(data_bottin.fillna(''))

Création d'un nouveau dataframe contenant name, job, street_clean, number_clean et l'entrée Wikipast en texte

In [ ]:
df = pd.DataFrame()
df['page_text'] = "* "+"[["+data_bottin['year'].astype(str)+"]]"\
+" / [[Paris]]. "\
+"[["+data_bottin['name'].astype(str)+"]]"\
+", [["+ data_bottin['job'].astype(str)+"]]"\
+", exerce son activité au"\
+" "+data_bottin['number_clean'].astype(str)+" "\
+"[["+data_bottin['street_clean'].astype(str)+"]]."\
+" ["+data_bottin['url'].astype(str)+"]"

In [ ]:
df['name'] = data_bottin['name']
df['job'] = data_bottin['job']
df['street_clean'] = data_bottin['street_clean']
df['number_clean'] = data_bottin['number_clean']

In [ ]:
df = df.reset_index()[['page_text','name','job','street_clean','number_clean']]
df.info()

In [ ]:
pd.set_option('display.max_colwidth', None)
df.head()

Groupement des entrées ayant les mêmes nom, job, street_clean et number_clean en mettant les textes générés en une liste

In [ ]:
df_grpby = df.groupby(['name','job','street_clean','number_clean'])['page_text'].apply(list)
df1 = df_grpby.to_frame().reset_index()
df1.info()

Filtrer les lignes ayant une seule entrée Wikipast et dont le name ne commence pas par une lettre de l'alphabet

In [ ]:
df1 = df1[df1['page_text'].apply(lambda x: len(x)>1)].copy()
df1 = df1[df1['name'].apply(lambda x: x[0].isalpha())].copy()
df1 = df1.reset_index()[['name','job','street_clean','number_clean','page_text']]
df1.info()

Concaténation des entrées Wikipast pour former un texte qui sera écrit sur les pages

In [ ]:
df1['page_text'] = df1['page_text'].apply(lambda l: "\n".join(sorted(l)))

Génération de noms pour les homonymes

In [ ]:
clean_name = df1['name'].astype(str)\
+" ("+ df1['job'].astype(str)+","\
+" "+df1['number_clean'].astype(str)\
+" "+df1['street_clean'].astype(str)+")"

df1['name_page'] = df1['name']
df1.loc[df1.duplicated(['name'], False), 'name_page'] = clean_name

Création de dataframe avec name: le nom partagée, et name_page: les noms des pages des homonymes

In [ ]:
duplicateRowsDF = df1[df1.duplicated(['name'], False)].copy()
#df2 = df1.duplicated(['name'], False), ['name']).copy()
duplicateRowsDF['name_page'] = "* [[" + df1['name_page'].astype(str)+ "]]"
df_grpby2 = duplicateRowsDF.groupby(['name'])['name_page'].apply(list)
df2 = df_grpby2.to_frame().reset_index()
df2['name_page'] = df2['name_page'].apply(lambda l: "\n".join(sorted(l)))

# Ecriture sur Wikipast

In [ ]:
from pywikiapi import Site, ApiError

Fonction pour écrire les pages des personnes sur Wikipast. Retourne un dictionnaire avec les pages écrites, éditées et dont l'écriture/édition a échoué

In [ ]:
def write_page_to_wikipast(dataframe):
    status = {'fail': list(), 'create': list(), 'edit': list()}
    
    for index, row in dataframe.iterrows():
        #liste des noms de pages, doit provenir du DataFrame df1
        titre = row['name_page']

        entries = row['page_text']
        
        #verifier qu'on écrase pas une page
        try:  #La page n'existe pas on en crée une
            site('edit', title=titre,
                 section='new',
                 sectiontitle='Biographie',
                 text=entries,
                 bot='true',
                 token=site.token(),
                 createonly=True)
            status['create'].append(titre)

        except ApiError: 
            #La page existait déjà, on rajoute à a fin
            if row['name_page'] == row['name']:
                titre = row['name_page']\
                +" ("+ row['job']+","\
                +" "+row['number_clean']\
                +" "+row['street_clean']+")"
                
            try:
                site('edit', title=titre,
                     section='new',
                     sectiontitle='Biographie',
                     text=entries,
                     bot='true',
                     token=site.token(),
                     createonly=True)
                status['create'].append(titre)
            except ApiError:
                try:
                    site('edit', title=titre + add_homonymes,
                        appendtext='\n'+entries,
                        bot='true',
                        token=site.token())
                    status['edit'].append(titre)
                except:
                    status['fail'].append(titre)
                    print("Une autre erreur est survenue (sûrement une erreur réseau)")
            except:
                print("Une autre erreur est survenue (sûrement une erreur réseau)")
                status['fail'].append(titre)
        except:
            print("Une autre erreur est survenue (sûrement une erreur réseau)")
            status['fail'].append(titre)
    
    return status

Fonction pour écrire les pages des homonymes sur Wikipast. Retourne un dictionnaire avec les pages écrites, éditées et dont l'écriture/édition a échoué

In [ ]:
def write_homonym_page_to_wikipast(dataframe):
    status = {'fail': list(), 'create': list(), 'create_with_homonyme': list(), 'edit': list()}
    
    for index, row in dataframe.iterrows():
        #liste des noms de pages, doit provenir du DataFrame df1
        titre = row['name']

        entries = row['name_page']
        
        #verifier qu'on écrase pas une page
        try:  #La page n'existe pas on en crée une
            site('edit', title=titre,
                 section='new',
                 prependtext="Cette [[page d'homonymie]] recense différentes personnes partageant le même nom.\n",
                 sectiontitle="Liste des homonymes",
                 text=entries,
                 bot='true',
                 token=site.token(),
                 createonly=True)
            status['create'].append(titre)


        #La page existe déjà, on rajoute (Homonymes) dans le titre
        except ApiError: 
            try:  #La page n'existe pas on en crée une
                site('edit', title=titre+' (Homonymes)',
                     section='new',
                     prependtext="Cette [[page d'homonymie]] recense différentes personnes partageant le même nom.\n",
                     sectiontitle="Liste des homonymes",
                     text=entries,
                     bot='true',
                     token=site.token(),
                     createonly=True)
                status['create_with_homonyme'].append(titre)
            #La page existait déjà, on rajoute à la fin    
            except ApiError:
                try:
                    site('edit', title=titre+' (Homonymes)',
                        appendtext='\n'+entries,
                        bot='true',
                        token=site.token())
                    status['edit'].append(titre)
                except:
                    status['fail'].append(titre)
                    print("Une autre erreur est survenue (sûrement une erreur réseau)")
        except:
            status['fail'].append(titre)
            print("Une autre erreur est survenue (sûrement une erreur réseau)")
            #Une autre erreur est survenue (sûrement une erreur réseau)
            #devrait garder l'index d'entrées qui n'a pas réussi, pour ré-éssayer plus tard

    return status

Création du lien avec Wikipast et écriture des 32 premières entrées générées + la première entrée pour les pages homonymes (les 32 premières entrées couvrent en intégralité la première occurence dans la liste de noms d'homonymes)

In [ ]:
user = 'Clement.lhoste@BottinBot6'
password = 'q0fimgpdbef61a9lbkiumrhtpq2prmi8'

In [ ]:
site = Site('http://wikipast.epfl.ch/wikipast/api.php') # Définition de l'adresse de l'API
site.no_ssl = True # Désactivation du https, car pas activé sur wikipast
site.login(user, password) # Login du bot

In [ ]:
to_import = df1[41:10000]
to_import

In [ ]:
df2.head(1)

In [ ]:
#page_writing_res = write_page_to_wikipast(to_import)

In [ ]:
page_writing_res

In [ ]:
#homonym_writing_res = write_homonym_page_to_wikipast(df2.head(1))

In [ ]:
homonym_writing_res